In [ ]:
# trained on local machine - GTX 1060 3GB GRAM
# base code from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#training
# RNN inspiration from https://github.com/neka-nat/distributed_rl/tree/master/distributed_rl
#                 R2D2 paper: https://openreview.net/pdf?id=r1lyTjAqYX
# Extra DQN changes inspired from: https://github.com/fg91/Deep-Q-Learning/blob/master/DQN.ipynb
# Hyperparams from rainbow paper - https://arxiv.org/pdf/1710.02298.pdf


# https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.p
# Section containing OpenAI atari wrappers from their repository above

import numpy as np
import os
os.environ.setdefault('PATH', '')
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)



class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    # def __init__(self, env, width=84, height=84, grayscale=True, dict_space_key=None):
    def __init__(self, env, width=84, height=84, grayscale=True, dict_space_key=None):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        If the environment uses dictionary observations, `dict_space_key` can be specified which indicates which
        observation should be warped.
        """
        super().__init__(env)
        self._width = width
        self._height = height
        self._grayscale = grayscale
        self._key = dict_space_key
        if self._grayscale:
            num_colors = 1
        else:
            num_colors = 3

        new_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(self._height, self._width, num_colors),
            dtype=np.uint8,
        )
        if self._key is None:
            original_space = self.observation_space
            self.observation_space = new_space
        else:
            original_space = self.observation_space.spaces[self._key]
            self.observation_space.spaces[self._key] = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3

    def observation(self, obs):
        if self._key is None:
            frame = obs
        else:
            frame = obs[self._key]

        if self._grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self._width, self._height), interpolation=cv2.INTER_AREA
        )
        if self._grayscale:
            frame = np.expand_dims(frame, -1)

        if self._key is None:
            obs = frame
        else:
            obs = obs.copy()
            obs[self._key] = frame
        return obs


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-1)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

    def count(self):
        frames = self._force()
        return frames.shape[frames.ndim - 1]

    def frame(self, i):
        return self._force()[..., i]

def make_atari(env_id,video_every, max_episode_steps=None):
    env = gym.make(env_id)

    # assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    if max_episode_steps is not None:
        pass
        # env = TimeLimit(env, max_episode_steps=max_episode_steps)
    # Added video Monitor
    if video_every!=0:
        env = gym.wrappers.Monitor(env, "./recordings/" + env_id + '/', video_callable=lambda episode_id: (episode_id % video_every) == 0, force=True)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env


In [ ]:

import gym
import gym.wrappers.monitoring.video_recorder

import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from  collections import deque
from collections import Counter
from PIL import Image

# import atari_wrappers
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from tqdm.auto import tqdm

torch.cuda.empty_cache()

In [ ]:
# env = gym.make('Breakout-v4')#.unwrapped
# env = gym.make('Gravitar-v0').unwrapped
# game = 'BreakoutNoFrameskip-v4'
game = 'GravitarNoFrameskip-v4'
# game= 'PongNoFrameskip-v0'

In [ ]:
# Wrappers
video_every=5
env = make_atari(game,video_every,max_episode_steps=None)
# env = gym.wrappers.Monitor(env, "./recordings/"+game+'/', video_callable=lambda episode_id: (episode_id%video_every)==0,force=True)
# env = atari_wrappers.wrap_deepmind(env,episode_life=True,clip_rewards=False,frame_stack=True)
env = wrap_deepmind(env,episode_life=False,clip_rewards=False,frame_stack=True)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Replay memory class, storing states, actions, following state and the reward gained

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))
Sequence = namedtuple('Sequence',('transitions','recurrent_state'))

class ReplayMemory(object):

    def __init__(self, capacity, files):
        self.files = files
        self.currNewestFile=0
        self.fileName='./memory/memory-data'#+file+'.pt'
        self.liveFiles = set()
        self.capacity = capacity
        self.memory = []
        self.position = 0
        self.priorities = np.zeros(capacity)
        self.alpha = 0.6
        load = False
        if load:
            self.load()


    def push(self, seq):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = seq
        self.priorities[self.position] = 1
        self.position = (self.position + 1) % self.capacity
        if self.position==0:
            self.save()

    def save(self):

        try:
            for i in range(0,capacity,1000):
                memToSave = {'mem': self.memory[i:i+1000],'pri':self.priorities[i:i+1000]}
                torch.save(memToSave,self.fileName+str(i)+'.pt')
        except:
            print('Error While Saving')
            pass
    def load(self):
        try:
            for i in range(0,capacity,1000):
                TensorDict = torch.load(self.fileName+str(i)+'.pt')
                self.memory+=TensorDict['mem']
                self.priorities[i:i+1000] = TensorDict['pri']
        except:
            print('Error While Saving')
            pass

        self.position = len(self.memory) % self.capacity

    def memSample(self):
        probs = np.array(list(range(1,len(self.liveFiles)+2)))
        probs[-1]*=2
        probs= probs / probs.sum()

        experiences = np.random.choice(len(self.liveFiles)+1,BATCH_SIZE,p=probs)
        experiences = self.calcFreq(experiences)

        data = []
        for i in experiences:
            # experiences[i] (batch size from that file), i (index of file)
            if i==len(self.liveFiles):
                data+= self.sample(experiences[i])
            else:
                x = (self.currNewestFile-i-1)%self.files
                data+= self.loadFile(self.fileName+str(x)+'.pt',experiences[i])
        return data


    def calcFreq(self,data):
        freq = {}
        for i in data:
            if i in freq:
                freq[i]+=1
            else:
                freq[i]=1
        return freq

    def loadFile(self,name,batch_size):
        return random.sample(torch.load(name),batch_size)

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def PERSample(self,batch_size, beta =0.6):
        # beta changes over time
        #Can change to use sum-tree
        if len(self.memory) < self.capacity:
            tmpPriorities = self.priorities[:self.position]
        else:
            tmpPriorities = self.priorities

        #priority_j ^alpha / sum priorities
        probs = tmpPriorities**self.alpha
        probs /= probs.sum()

        indices = np.random.choice(len(self.memory), batch_size, p=probs)
        tmpPriorities = tmpPriorities[indices]
        loss_weights = (len(tmpPriorities)*tmpPriorities/len(self.memory))**(-beta)
        loss_weights /= loss_weights.max()

        return [self.memory[index] for index in indices], indices, loss_weights

    def updatePriorities(self, indices, newPriorities):
        for i in range(len(indices)):
            self.priorities[indices[i]] = newPriorities[i]

    def __len__(self):
        return len(self.memory)

In [ ]:
# https://github.com/Shmuma/ptan/blob/master/samples/rainbow/lib/dqn_model.py
# Factorised Noisy Layer
class NoisyFactorizedLinear(nn.Linear):
    """
    NoisyNet layer with factorized gaussian noise
    N.B. nn.Linear already initializes weight and bias to
    """
    def __init__(self, in_features, out_features, sigma_zero=0.4, bias=True):
        super(NoisyFactorizedLinear, self).__init__(in_features, out_features, bias=bias)
        sigma_init = sigma_zero / math.sqrt(in_features)
        self.sigma_weight = nn.Parameter(torch.Tensor(out_features, in_features).fill_(sigma_init))
        self.register_buffer("epsilon_input", torch.zeros(1, in_features))
        self.register_buffer("epsilon_output", torch.zeros(out_features, 1))
        if bias:
            self.sigma_bias = nn.Parameter(torch.Tensor(out_features).fill_(sigma_init))

    def forward(self, input):
        torch.randn(self.epsilon_input.size(), out=self.epsilon_input)
        torch.randn(self.epsilon_output.size(), out=self.epsilon_output)

        func = lambda x: torch.sign(x) * torch.sqrt(torch.abs(x))
        eps_in = func(self.epsilon_input)
        eps_out = func(self.epsilon_output)

        bias = self.bias
        if bias is not None:
            bias = bias + self.sigma_bias * torch.autograd.Variable(eps_out.t())
        noise_v = torch.autograd.Variable(torch.mul(eps_in, eps_out))
        return F.linear(input, self.weight + self.sigma_weight * noise_v, bias)


In [ ]:
# Number of Linear input connections depends on output of conv2d layers
# and therefore the input image size, so compute it.
def conv2d_size_out(size, kernel_size = 5, stride = 2):
    return (size - (kernel_size - 1) - 1) // stride  + 1


class DuelingLSTMDQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DuelingLSTMDQN, self).__init__()
        if scaledScreenSize[1] > 80:
            conv_out = 64
            # Hyper parameters from Rainbow paper - https://arxiv.org/pdf/1710.02298.pdf
            if grey:
                self.conv1 = nn.Conv2d(1*4, 32, kernel_size=8, stride=4)
            else:
                self.conv1 = nn.Conv2d(3*4, 32, kernel_size=8, stride=4)
            self.bn1 = nn.BatchNorm2d(32)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
            self.bn2 = nn.BatchNorm2d(64)
            self.conv3 = nn.Conv2d(64, conv_out, kernel_size=3, stride=1)
            self.bn3 = nn.BatchNorm2d(conv_out)
            convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w,kernel_size=8,stride=4),kernel_size=4),kernel_size=3,stride=1)
            convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h,kernel_size=8,stride=4),kernel_size=4),kernel_size=3,stride=1)

        else: # scaledScreenSize[0] == 64:
            # for reduced model complexity
            conv_out = 32
            if grey:
                self.conv1 = nn.Conv2d(1*4, 16, kernel_size=6, stride=3)
            else:
                self.conv1 = nn.Conv2d(3*4, 16, kernel_size=6, stride=3)
            self.bn1 = nn.BatchNorm2d(16)
            self.conv2 = nn.Conv2d(16, 32, kernel_size=4, stride=2)
            self.bn2 = nn.BatchNorm2d(32)
            self.conv3 = nn.Conv2d(32, conv_out, kernel_size=3, stride=1)
            self.bn3 = nn.BatchNorm2d(conv_out)
            convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w,kernel_size=6,stride=3),kernel_size=4),kernel_size=3,stride=1)
            convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h,kernel_size=6,stride=3),kernel_size=4),kernel_size=3,stride=1)


        lstm_input_size = convw * convh * conv_out
        self.linear_input_size = 512
        # linear_input_size = convw * convh * conv_out


        self.lstm = nn.LSTMCell(lstm_input_size, self.linear_input_size)
        self.hiddenStatesBatch = torch.zeros(BATCH_SIZE,self.linear_input_size).to(device)
        self.cellStatesBatch = torch.zeros(BATCH_SIZE,self.linear_input_size).to(device)
        self.hiddenStatesLive = torch.zeros(1,self.linear_input_size).to(device)
        self.cellStatesLive = torch.zeros(1,self.linear_input_size).to(device)

        # dualing networks
        # https://arxiv.org/pdf/1511.06581.pdf
        # self.adv1 = NoisyFactorizedLinear(self.linear_input_size, 512)
        # self.adv2 = NoisyFactorizedLinear(512, outputs)
        # self.val1 = NoisyFactorizedLinear(self.linear_input_size, 512)
        # self.val2 = NoisyFactorizedLinear(512, 1)

        self.adv1 = nn.Linear(self.linear_input_size, 512)
        self.adv2 = nn.Linear(512, outputs)
        self.val1 = nn.Linear(self.linear_input_size, 512)
        self.val2 = nn.Linear(512, 1)
        # weight initialisation https://towardsdatascience.com/tutorial-double-deep-q-learning-with-dueling-network-architectures-4c1b3fb7f756
        nn.init.kaiming_normal_(self.adv1.weight)
        nn.init.kaiming_normal_(self.val1.weight)

    def reset(self,done, batch=False):
        if batch:
            self.hiddenStatesBatch.detach()
            self.cellStatesBatch.detach()
            if done:
                self.hiddenStatesBatch = self.hiddenStatesBatch.zero_()
                self.cellStatesBatch = self.hiddenStatesBatch.zero_()
        else:
            self.hiddenStatesLive.detach()
            self.cellStatesLive.detach()
            if done:
                self.hiddenStatesLive = self.hiddenStatesLive.zero_()
                self.cellStatesLive = self.hiddenStatesLive.zero_()

    def getState(self, batch=False):
        if batch:
            return self.hiddenStatesBatch.detach().clone().cpu(), self.cellStatesBatch.detach().clone().cpu()
        else:
            return self.hiddenStatesLive.detach().clone().cpu(), self.cellStatesLive.detach().clone().cpu()

    def setState(self, state,device, batch=False):
        if batch:
            hiddens, cells = state
            self.hiddenStatesBatch = hiddens.clone().to(device)
            self.cellStatesBatch = cells.clone().to(device)
        else:
            hiddens, cells = state
            self.hiddenStatesLive = hiddens.clone().to(device)
            self.cellStatesLive = cells.clone().to(device)

    # Called with either one element to determine next action, or a batch
    # during optimisation.
    def forward(self, x, hiddenState=None, cellState=None):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.contiguous().view(x.size(0), -1)
        if not(hiddenState==None):

            self.hiddenStatesLive, self.cellStatesLive = self.lstm(x,(hiddenState,cellState))
            adv = F.relu(self.adv1(self.hiddenStatesLive))
            val = F.relu(self.val1(self.hiddenStatesLive))

        else:
            self.hiddenStatesBatch, self.cellStatesBatch = self.lstm(x,(self.hiddenStatesBatch,self.cellStatesBatch))
            adv = F.relu(self.adv1(self.hiddenStatesBatch))
            val = F.relu(self.val1(self.hiddenStatesBatch))
        # Duelling architecture
        adv = self.adv2(adv)
        val = self.val2(val)
        return val + adv - adv.mean(1, keepdim=True)

    def calcLoss(self,batch):
        copy = DuelingLSTMDQN(screen_height, screen_width, n_actions).to(device)
        copy.load_state_dict(self.state_dict())
        copy.eval()
        batch = Sequence(*zip(*batch))
        batch = Sequence(list(zip(*batch.transitions)),list(zip(*batch.recurrent_state)))

        hiddenStates = torch.cat(batch.recurrent_state[0])
        cellStates = torch.cat(batch.recurrent_state[1])


        self.setState((hiddenStates,cellStates),device,True)
        target_net.setState((hiddenStates,cellStates),device,True)

        #burn in the model
        with torch.no_grad():
            for t in range(n_burn_in):
                tmpTransition = Transition(*zip(*batch.transitions[t]))
                state = get_screen(tmpTransition.state,True).to(device)
                self.forward(state)
                target_net.forward(state)
            copy.setState(self.getState(True),device,True)
            for t in range(n_burn_in,n_burn_in+n_step_return):
                tmpTransition = Transition(*zip(*batch.transitions[t]))
                state = get_screen(tmpTransition.state,True).to(device)
                copy.forward(state)
                target_net.forward(state)
        self.reset(False,True)

        # init loss array
        delta = torch.zeros(len(batch.transitions)-n_burn_in-n_step_return,BATCH_SIZE,1,device=device)
        with torch.set_grad_enabled(True):
            for t in range(n_burn_in,len(batch.transitions)-n_step_return):
                tmpTransition = Transition(*zip(*batch.transitions[t]))
                nextTransition = Transition(*zip(*batch.transitions[t+n_step_return]))
                state_batch = get_screen(tmpTransition.state,True).to(device)
                action_batch = torch.cat(tmpTransition.action).to(device)
                reward_batch = torch.stack(tmpTransition.reward).to(device)
                next_state_batch = get_screen(nextTransition.state,True).to(device)
                done_batch = torch.stack(nextTransition.done).to(device)

                state_action_values = self.forward(state_batch)
                state_action_values = state_action_values.gather(1,action_batch)
                actionPrediction = copy.forward(next_state_batch).argmax(dim=1).unsqueeze(1).detach()

                next_state_values = target_net(next_state_batch).gather(1, actionPrediction).detach()

                expected_state_action_values = (self.inv_value_function(next_state_values) * GAMMA * (1.0 - done_batch)) \
                                           + reward_batch
                expected_state_action_values = self.value_function(expected_state_action_values)

                # delta[t-n_burn_in] = F.smooth_l1_loss(state_action_values,expected_state_action_values.float(),reduction='none')
                delta[t-n_burn_in] = F.mse_loss(state_action_values,expected_state_action_values.float(),reduction='none')

        # 0.9 is eta value in R2 paper
        priorities = 0.9 * delta.max(dim=0)[0] + (1.0 - 0.9) * delta.mean(dim=0)
        return delta.sum(dim=0), priorities.detach()
        # return delta.pow(2).sum(dim=0), priorities.detach()

    # https://arxiv.org/pdf/1805.11593.pdf
    def value_function(self,x,eps=1e-3):
        return x.sign() * ((x.abs()+1).sqrt()-1)+eps*x
    def inv_value_function(self,x,eps=1e-3):
        return x.sign() * ((((1+4*eps*(x.abs()+1+eps)).sqrt()-1) / (2 *eps)).pow(2)-1)



In [ ]:
scaledScreenSize = (84,84)
grey=True

def get_screen(screen,batch=False):

    temp = np.ascontiguousarray(screen, dtype=np.float32) / 255
    # BATCH,x,y,frame
    if batch:
        return torch.from_numpy(temp).permute(0,3,1,2)
    else:
        return torch.from_numpy(temp).unsqueeze(0).permute(0,3,1,2)

# output example image

# temp = env.reset()
# env.step(0)
# env.step(0)
# plt.figure()
# if grey:
#     plt.imshow(get_screen(temp).cpu().permute(2,0,1)[0].numpy(),
#             interpolation='none', cmap='gray')
# else:
#     plt.imshow(get_screen(temp).cpu().permute(1, 2, 0).numpy(),
#             interpolation='none')
# plt.title('What the model Sees')
# plt.show()
#
# exit()

In [ ]:
num_episodes = 10000
BATCH_SIZE = 23 # small batch due to training on GTX 1060 3GB
GAMMA = 0.999**3
TARGET_UPDATE = 1000 # frames
POLICY_UPDATE = 24 # frames
BETA_DECAY = 2000
OUTPUT_FREQUENCY = 10
n_step_return = 5
n_sequence = 50 #size of LSTM
n_overlap = 25 #the next RNN state is half of the sequence length
n_burn_in = 25 #how many steps before RNN is set to actually update - ie gets better initial state

# update to 20 batch - 50 n_sequence, eps decay 150, eps max 0.4, episodic end of life = true -
# already made lr and loss power changes

# init with correct dims
if scaledScreenSize[1]==84:
    screen_width = 84
    screen_height = 84
else:
    screen_width = 64
    screen_height = 64

# Get number of actions from gym action space
n_actions = env.action_space.n

# Actor generates experiences
# Learner learns from experiences every episode
# params = torch.load('./memory/target_net.chkpt')
Actor_net = DuelingLSTMDQN(screen_height, screen_width, n_actions).to(device)
Learner_net = DuelingLSTMDQN(screen_height, screen_width, n_actions).to(device)
# Learner_net.load_state_dict(params['A'])
target_net = DuelingLSTMDQN(screen_height, screen_width, n_actions).to(device)
# target_net.load_state_dict(Learner_net.state_dict())
target_net.eval()





optimiser = optim.Adam(Learner_net.parameters(),lr=1e-4,eps=1e-3)
# optimiser.load_state_dict(params['optimiser'])
capacity = 10000
files = 5
memory = ReplayMemory(capacity=capacity,files=files)


def select_action(state,n_episode, LSTMState):


    # global steps_done
    sample = random.random()
    # custom eps greedy function
    epsilon = max( abs(math.exp(-(n_episode/400))*0.4*math.sin(n_episode/10)), 0.001)

    if sample > epsilon:
        with torch.no_grad():
            x = Actor_net(get_screen(state).to(device),LSTMState[0].to(device),LSTMState[1].to(device))
            return x.max(1)[1].view(1, 1).cpu()
    else:
        return torch.tensor([[random.randrange(n_actions)]], device='cpu', dtype=torch.long)



In [ ]:

def optimize_model(n_episode):

    # beta = 0.75
    beta = min(1, 0.4 + 1*(n_episode/BETA_DECAY))

    if len(memory) < BATCH_SIZE*2:
        return
    lossArr = []
    for _ in range(2):

        Learner_net.reset(done=True,batch=True)


        # transitions = memory.sample(BATCH_SIZE)
        transitions, indices, loss_weights = memory.PERSample(BATCH_SIZE,beta)
        # transitions, indices = memory.PERSample(BATCH_SIZE)
        # transitions = memory.memSample()

        delta,priorities = Learner_net.calcLoss(transitions)

        memory.updatePriorities(indices,priorities)
        loss = (delta*torch.Tensor(loss_weights).to(device)).mean().float()
        # loss = (delta).mean().float()
        lossArr.append(loss.clone().detach().cpu().numpy())
        # Optimise the model
        optimiser.zero_grad()
        loss.backward()
        # clip grads
        for param in Learner_net.parameters():
            param.grad.data.clamp_(-1, 1)
        optimiser.step()
    return lossArr

In [ ]:
import time
#Training
start=time.time()
seed = 742
torch.manual_seed(seed)
env.seed(seed)
random.seed(seed)
np.random.seed(seed)
env.action_space.seed(seed)

marking=[]
episodeScores=[]
losses = []
stepBuffer = deque(maxlen=n_step_return)
#precalculate GAMMA n step return values
gamma_nsteps = [GAMMA ** i for i in range(n_step_return+1)]
sequenceBuffer = []
recurrentBuffer = []
frames = 0
n_episode=2300

score=0
actorUpdates=0
# Initialize the environment and state
maxLives = env.ale.lives()
state = env.reset()
lives = maxLives
with tqdm() as pbar:
    while n_episode<num_episodes:
    # for n_episode in tqdm(range(num_episodes+1)):

        sequenceBuffer = []
        recurrentBuffer = []
        while True:

            recurrentBuffer.append(Actor_net.getState())
            action = select_action(state,n_episode,recurrentBuffer[-1])

            obvs, reward, done, info = env.step(action.item())
            score+=reward

            # Reward clipping and negative reward on death
            if info['ale.lives'] != lives:
                pbar.update(1)
                pbar.set_postfix({'episode':n_episode,'score':score,'Actor Updates':actorUpdates,'Frames':frames})
                reward-=1
                lives = info['ale.lives']
            # reward = np.clip(reward, -1, 1)

            reward = torch.tensor([reward])

            # Observe new state
            next_state = obvs
            if done:
                next_state = None

            stepBuffer.append(Transition(state,action,next_state,reward,torch.tensor([float(done)])))
            if len(stepBuffer) == stepBuffer.maxlen:
                reward_nstep = sum([gamma_nsteps[-(i+2)] * stepBuffer[i].reward for i in range(n_step_return)])
                sequenceBuffer.append(Transition(stepBuffer[0].state,stepBuffer[0].action,stepBuffer[0].next_state,reward_nstep,torch.tensor([float(done)])))
            if (len (sequenceBuffer)) == n_sequence+n_burn_in:
                memory.push(Sequence(sequenceBuffer,recurrentBuffer[0]))
                #chop sequence and recurrent buffers
                sequenceBuffer = sequenceBuffer[-(n_burn_in+n_overlap):]
                recurrentBuffer = recurrentBuffer[-(n_burn_in+n_overlap+n_step_return):]
            elif done and len(sequenceBuffer) > n_burn_in+n_overlap+n_step_return:
                # case of we're done so we push the rest of the sequence to memory
                memory.push(Sequence(sequenceBuffer,recurrentBuffer[0]))
            if frames % POLICY_UPDATE == 0:
                x = optimize_model(n_episode)
                if x != None:
                    losses += x
            # Update the target network, copying all weights and biases in DQN
            if frames % TARGET_UPDATE == 0:
                actorUpdates+=1
                Actor_net.load_state_dict(Learner_net.state_dict())
                target_net.load_state_dict(Learner_net.state_dict())

            # Move to the next state
            state = next_state
            frames+=1
            if done:
                stepBuffer.clear()

                break

        state = env.reset()
        lives = env.ale.lives()
        if lives == maxLives:
            episodeScores.append(int(score))
            marking.append(score)

            # do not change lines 44-48 here, they are for marking the submission log
            if n_episode%100 == 0:
                print("marking, episode: {}, score: {:.1f}, mean_score: {:.2f}, std_score: {:.2f}".format(
                    n_episode, score, np.array(marking).mean(), np.array(marking).std()))
                marking = []

            if n_episode % OUTPUT_FREQUENCY == 0:
                if len(marking)>0:
                    print("episode: {}, mean_last30: {:.2f}, mean_score: {:.2f}, std_score: {:.2f}, mean_loss: {:.4f}".format(
                        n_episode, np.array(episodeScores[-30:]).mean(), np.array(marking).mean(), np.array(marking).std(),np.array(losses[-20:]).mean()),episodeScores[-OUTPUT_FREQUENCY:])
            n_episode+=1
            score=0





print('Complete')
# env.render()
env.close()

In [ ]:
memory.save()
torch.save({'A':target_net.state_dict(), 'optimiser':optimiser.state_dict(),}, './memory/target_net.chkpt')

In [ ]:
# Trend Graphs
episodeScoresEnum = sorted(enumerate(episodeScores),reverse=True,key=lambda x:x[1])
print('Top 5 scores overall')
print(episodeScoresEnum[:5])
tempScores = sorted([(i,j) for i,j in enumerate(episodeScores) if i%5==0],key=lambda x:x[1],reverse=True)
print('Top videos:')
print(tempScores[:10])
print('Bottom Videos')
print(tempScores[-10:][::-1])
print('Total Frames Trained on: ',frames)



In [ ]:
averages30 = []
averages = []

for i in range(len(losses)):
    averages.append(np.array(losses[max(i-100,0):i+1]).mean())
    averages30.append(np.array(losses[max(i-30,0):i+1]).mean())

plt.plot(range(len(losses)),losses,alpha=0.4,label='actual')
plt.plot(range(len(averages)),averages,label='100 average')
plt.plot(range(len(averages30)),averages30,label='30 average',alpha=0.6)
plt.legend(loc='upper left')
plt.ylabel('Average loss')
plt.yscale('log')
plt.xlabel('loss Updates')

plt.show()

In [ ]:
averages30 = []
averages = []

for i in range(len(episodeScores)):
    averages.append(np.array(episodeScores[max(i-100,0):i+1]).mean())
    averages30.append(np.array(episodeScores[max(i-30,0):i+1]).mean())

plt.figure(1,(10,6),1000)
plt.plot(range(len(episodeScores)),episodeScores,alpha=0.4,label='actual',linewidth=0.5)
plt.plot(range(len(averages)),averages,label='100 average')
plt.plot(range(len(averages30)),averages30,label='30 average',alpha=0.6)
plt.legend(loc='upper left')
plt.ylabel('Average score')
plt.xlabel('episodes')

plt.show()